<a href="https://colab.research.google.com/github/shireesha27/demo/blob/master/DLQuestion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing neccessary libraries
# Python program using numpy for arranging a given set of data into a table
# Data processing to read CSV file 
import pandas as pd 
# Keras is the neural network library
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# From keras.models import sequential model
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# Python program using Matplotib for forming a linear plot
from matplotlib import pyplot
# Importing the library for splitting the data
from sklearn.model_selection import train_test_split
# Importing one hot encoding library
from keras.utils.np_utils import to_categorical
import re
# importing preprocessing library
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Reading the data from sentiment.csv file
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]


In [ ]:
# This syntax is using to the data which has unnecessary things from text and convert them to lowercase
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [ ]:
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

In [ ]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
# Tokenizing the data and convert the text to sequences.
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

In [ ]:
# Addition of padding to ensure that all the sequences have the same shape.
X = pad_sequences(X)
print(X)
embed_dim = 128
lstm_out = 196

In [ ]:
# Function to create a model
def createmodel():
    # Creating a model
    model = Sequential()
    # embed_dim, lstm_out, batch_size, droupout_x variables are hyperparameters.
    # these parameters helps to achieve good results
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    # Compiling the model
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    print(model.summary())

In [ ]:
# LabelEncoder to encode the text into categorical integer values
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
# One-hot-encoder 
# utils.to_categorical is used to convert array of labeled data to one-hot vector
y = to_categorical(integer_encoded)
# Splitting the dataset into test and train data
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape) 
print(X_test.shape,Y_test.shape) 


In [ ]:
batch_size = 40
model = createmodel()
# Model.fit() using to padded training data and validating the data.
# epoch: Number of times the learning algorithm will work through the entire training data set
history= model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
# Evaluating the model
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=0)
# Defining the grid search parameters
batch_size = 40
epochs = [1, 2, 3]
param_grid = dict(epochs=epochs)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, Y_train, batch_size = 40)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# By seeing above result the batch size of 40 and epochs number 2 achieved the best result of about 68% accuracy.

In [ ]:
# Bonus question
# Printing the keys from history.
print(history.history.keys())
# Analyze by visualizing data.
# Summarize history for an accuracy.
pyplot.plot(history.history['accuracy'])
pyplot.title('Model accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train'], loc= 'upper left')
pyplot.show()


In [ ]:
# Summarize history for the loss.
pyplot.plot(history.history['loss'])
pyplot.title('Model loss')
pyplot.xlabel('epoch')
pyplot.ylabel('Loss')
pyplot.legend(['train'], loc= 'upper left')
pyplot.show()